# Agents, supercharged - Multi-agents, External tools, and more

In [ ]:
!pip install transformers[agents]

## Multi-agents

Multi-agent has been introduced in Microsoft's framework Autogen. It simply means having several agents working together to solve our task instead of only one.

For many tasks, rather than using a do-it-all system, we would prefer to specialize units on sub-tasks.

We can build hierarchical multi-agent systems with `transformers.agents`.

To do so, encapsulate the agent in a `ManagedAgent` object. This object needs arguments `agent`, `name`, and a `description`, which will then be embedded in the manager agent's system prompt to let it know how to call this managed agent, as we also do for tools.

Here is an example of making an agent that managed a specific web search agent using our `DuckDuckGoSearchTool`:

In [ ]:
from transformers.agents import ReactCodeAgent, HfApiEngine, DuckDuckGoSearchTool, ManagedAgent

llm_engine = HfApiEngine()

web_agent = ReactCodeAgent(tools=[DuckDuckGoSearchTool()],
                           llm_engine=llm_engine)

managed_web_agent = ManagedAgent(
    agent=web_agent,
    name='web_search',
    description='Runs web searches for us. Give it our query as an argument.'
)

manager_agent = ReactCodeAgent(tools=[],
                               llm_engine=llm_engine,
                               managed_agents=[managed_web_agent])

manager_agent.run('Who is the CEO of Hugging Face?')

## Advanced tool usage

### Directly define a tool by subclassing Tool, and share it to the Hub

We can build our tool following the fine-grained method: building a class that inherits from the `Tool` superclass.

The custom tool needs:
* an attribute `name`, which corresponds to the name of the tool itself. The name usually describes what the tool does. Since the code returns the model with the most downloads for a task, we can call it `model_download_counter`.
* an attribute `description` is used to populate the agent's system prompt.
* an `inputs` attribute, which is a dictionary with keys `"type"` and `"description"`. It contains information that helps the Python interpreter make decuated choices about the input.
* An `output_type` attribute, which specifies the output type.
* A `forward` method which contains the inference code to be executed.

The types for both `inputs` and `output_type` should be amongst Pydantic formats.

In [ ]:
from transformers import Tool
from huggingface_hub import list_models

class HFModelDownloadsTool(Tool):
    name = 'model_download_counter'
    description = """
        This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub.
        It returns the name of the checkpint.
    """

    inputs = {
        'task': {
            'type': 'string',
            'description': "the task category (such as text-classification, depth-estimation, etc)"
        }
    }
    output_type = 'string'

    def forward(self, task: str):
        model = next(iter(list_models(filter=task, sort='downloads', direction=-1)))
        return model.id

Now that the custom `HfModelDownloadsTool` class is ready, we can save it to a file named `model_downloads.py` and import it for use:

In [ ]:
from model_downloads import HFModelDownloadsTool

tool = HFModelDownloadsTool()

Load the tool with the `Tool.load_tool` function and pass it to the `tools` parameter in our agent:

In [ ]:
from transformers import load_tool, CodeAgent

model_download_tool = load_tool('hf-model-downloads')

Assuming that we have push this tool to the hub and named as `"<username>/hf-model-downloads"`.

### Use LangChain tools

To import a tool from LangChain, use the `from_langchain()` method. Here is how we can use it to recreate the intro's search result using a LangChain web search tool:

In [ ]:
from langchain.agents import load_tools
from transformers import Tool, ReactCodeAgent

search_tool = Tool.from_langchain(load_tools(['serpapi'])[0])

agent = ReactCodeAgent(tools=[search_tool])

agent.run("How many more blocks (also denoted as layers) in BERT base encoder than the encoder from the architecture proposed in Attention is All You Need?")